# OSM Online Loader

In [ ]:
from functional import seq
from srai.loaders.osm_loaders import OSMOnlineLoader
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS, HEX2VEC_FILTER
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from srai.plotting.folium_wrapper import plot_regions
from srai.regionalizers import geocode_to_region_gdf

## Filters
Filters are dictionaries used for specifying what type of objects one would like to download from OpenStreetMap.  
There is currently one predefined filter (from Hex2Vec paper) and one way to download a filter - using popular tags from taginfo API.  
They can also be defined manually in code.

Additionally, few predefined grouped filters are available (eg. `BASE_OSM_GROUPS_FILTER` and `GEOFABRIK_LAYERS`).
Grouped filters categorize base filters into groups.

### Get popular tags from taginfo API

In [ ]:
all_popular_tags = get_popular_tags()
num_keys = len(all_popular_tags)
num_values = seq(all_popular_tags.values()).map(len).sum()
f"Unique keys: {num_keys}. Key/value pairs: {num_values}"

In [ ]:
seq(all_popular_tags.items()).take(10).dict()

In [ ]:
frequent_in_wiki_only_tags = get_popular_tags(in_wiki_only=True, min_fraction=0.001)
frequent_in_wiki_only_tags

### Import a predefined filter

In [ ]:
hex_2_vec_keys = len(HEX2VEC_FILTER)
hex_2_vec_key_values = seq(HEX2VEC_FILTER.values()).map(len).sum()
f"Unique keys: {hex_2_vec_keys}. Key/value pairs: {hex_2_vec_key_values}"

In [ ]:
geofabrik_layers_keys = len(GEOFABRIK_LAYERS)
geofabrik_layers_key_values = (
    seq(GEOFABRIK_LAYERS.values())
    .flat_map(lambda filter: filter.items())
    .map(len)
    .sum()
)
print(
    f"Unique groups: {geofabrik_layers_keys}."
    f" Key/value pairs: {geofabrik_layers_key_values}"
)

## Using OSMOnlineLoader to download data for a specific area

### Download all parks in Wrocław, Poland

In [ ]:
loader = OSMOnlineLoader()
parks_filter = {"leisure": "park"}
wroclaw_gdf = geocode_to_region_gdf("Wrocław, Poland")
parks_gdf = loader.load(wroclaw_gdf, parks_filter)
parks_gdf

In [ ]:
folium_map = plot_regions(
    wroclaw_gdf, colormap=["lightgray"], tiles_style="CartoDB positron"
)
parks_gdf.explore(m=folium_map, color="forestgreen")

### Download hotels, bars, cafes, pubs and sport related objects in Barcelona

Uses grouped filters as an example.

In [ ]:
barcelona_gdf = geocode_to_region_gdf("Barcelona")
barcelona_filter = {
    "tourism": {"building": "hotel", "amenity": ["bar", "cafe", "pub"]},
    "sport": {"sport": "soccer", "leisure": ["pitch", "sports_centre", "stadium"]},
}

barcelona_objects_gdf = loader.load(barcelona_gdf, barcelona_filter)
barcelona_objects_gdf

### Tourism group

In [ ]:
folium_map = plot_regions(
    barcelona_gdf, colormap=["lightgray"], tiles_style="CartoDB positron"
)
barcelona_objects_gdf.query("tourism.notna()").explore(
    m=folium_map,
    color="orangered",
    marker_kwds=dict(radius=1),
)